<a href="https://colab.research.google.com/github/rubenjesus02002/Project-9-Data-Warehouse-setup-for-CME-data/blob/main/extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install azure-storage-blob
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 24.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [3]:
# read the config file that is json
import json
with open('config.json') as config_file:
  config = json.load(config_file)

# read information about source
SOURCE_CONNECTION_STRING = config['SOURCE_CONNECTION_STRING']
SOURCE_CONTAINER_NAME = config['SOURCE_CONTAINER_NAME']
SOURCE_FILE_NAME = config['SOURCE_FILE_NAME']

DESTINATION_CONNECTION_STRING = config['DESTINATION_CONNECTION_STRING']
DESTINATION_CONTAINER_NAME = config['DESTINATION_CONTAINER_NAME']
DESTINATION_FILE_NAME = config['DESTINATION_FILE_NAME']


In [4]:
# Assuming you have SOURCE_CONNECTION_STRING, SOURCE_CONTAINER_NAME,
# DESTINATION_CONNECTION_STRING, and DESTINATION_CONTAINER_NAME defined from your config files.

# Instantiate BlobServiceClient for the source
source_blob_service_client = BlobServiceClient.from_connection_string(SOURCE_CONNECTION_STRING)

# Get a client for the source container
source_container_client = source_blob_service_client.get_container_client(SOURCE_CONTAINER_NAME)

# List all blobs in the source container
blob_list = source_container_client.list_blobs()

# Loop through each blob in the source container
for blob in blob_list:
    print(f"Processing blob: {blob.name}")
    # Download the blob to a local file
    source_blob_name = blob.name
    # take only every after the forward slash
    local_file_name = source_blob_name.split('/')[-1]

    print(f"Downloading blob: {source_blob_name}")

    # Download the blob to a local file
    source_blob_client = source_blob_service_client.get_blob_client(container=SOURCE_CONTAINER_NAME, blob=source_blob_name)
    with open(local_file_name, "wb") as my_blob:
        download_stream = source_blob_client.download_blob()
        my_blob.write(download_stream.readall())

    print(f"Uploading blob: {source_blob_name}")

    # Upload the local file to the destination container
    destination_blob_service_client = BlobServiceClient.from_connection_string(DESTINATION_CONNECTION_STRING)
    destination_container_client = destination_blob_service_client.get_container_client(DESTINATION_CONTAINER_NAME)

    # You can use the same blob name or define a new one for the destination
    # destination_blob_name = source_blob_name
    with open(local_file_name, "rb") as data:
        destination_blob_client = destination_container_client.upload_blob(name=source_blob_name, data=data, overwrite=True)

    print(f"Uploaded blob '{source_blob_name}' to destination as '{source_blob_name}'.")

    # Optional: Clean up the local file
    import os
    os.remove(local_file_name)
    print(f"Removed local file: {local_file_name}")

print("Finished processing all blobs.")

Processing blob: 2024-12-02/XCBT_CommodityAgriculture.json
Uploading blob: 2024-12-02/XCBT_CommodityAgriculture.json
Uploaded blob '2024-12-02/XCBT_CommodityAgriculture.json' to destination as '2024-12-02/XCBT_CommodityAgriculture.json'.
Removed local file: XCBT_CommodityAgriculture.json
Processing blob: 2024-12-02/XCBT_Currency.json
Uploading blob: 2024-12-02/XCBT_Currency.json
Uploaded blob '2024-12-02/XCBT_Currency.json' to destination as '2024-12-02/XCBT_Currency.json'.
Removed local file: XCBT_Currency.json
Processing blob: 2024-12-02/XCEC_CommodityAgriculture.json
Uploading blob: 2024-12-02/XCEC_CommodityAgriculture.json
Uploaded blob '2024-12-02/XCEC_CommodityAgriculture.json' to destination as '2024-12-02/XCEC_CommodityAgriculture.json'.
Removed local file: XCEC_CommodityAgriculture.json
Processing blob: 2024-12-02/XCEC_Currency.json
Uploading blob: 2024-12-02/XCEC_Currency.json
Uploaded blob '2024-12-02/XCEC_Currency.json' to destination as '2024-12-02/XCEC_Currency.json'.
Rem